In [2]:
# First, import standard libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Tuple, Dict, List
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Then import sklearn before torch
from sklearn.preprocessing import StandardScaler

# Now import torch and its modules
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Finally import cuda specific modules
from torch.cuda import amp

def select_gpu():
    """Select the NVIDIA GPU if available."""
    if torch.cuda.is_available():
        # Get number of GPUs
        gpu_count = torch.cuda.device_count()
        print(f"\nFound {gpu_count} GPUs:")
        
        # Find NVIDIA GPU
        for i in range(gpu_count):
            gpu_properties = torch.cuda.get_device_properties(i)
            print(f"GPU {i}: {gpu_properties.name}")
            
            if 'NVIDIA' in gpu_properties.name:
                print(f"\nSelecting NVIDIA GPU: {gpu_properties.name}")
                torch.cuda.set_device(i)
                return torch.device(f'cuda:{i}')
    
    print("\nNo NVIDIA GPU found, using CPU")
    return torch.device('cpu')

def get_gpu_info():
    """Print GPU information if available."""
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            gpu_properties = torch.cuda.get_device_properties(i)
            print(f"\nGPU {i}: {gpu_properties.name}")
            print(f"Memory: {gpu_properties.total_memory / 1024**3:.2f} GB")
            print(f"CUDA Capability: {gpu_properties.major}.{gpu_properties.minor}")
    else:
        print("\nNo GPU available, using CPU")

class TemporalFeatureDataset(Dataset):
    """Dataset class with minimal output."""
    def __init__(self, features, targets, lookback=30, forecast_horizon=7):
        self.features = features
        self.targets = targets
        self.lookback = lookback
        self.forecast_horizon = forecast_horizon
        
        # Calculate valid indices
        self.valid_indices = len(features) - lookback - forecast_horizon + 1
        
        # Print only once during initialization
        print(f"\nDataset created with {self.valid_indices} samples")
        print(f"Lookback: {lookback}, Forecast horizon: {forecast_horizon}")
    
    def __len__(self):
        return self.valid_indices
    
    def __getitem__(self, idx):
        if idx < 0 or idx >= self.valid_indices:
            raise IndexError(f"Index {idx} out of bounds")
            
        x = self.features[idx:idx + self.lookback]
        y = self.targets[idx + self.lookback:idx + self.lookback + self.forecast_horizon]
        y = y.squeeze()
        
        return x, y

class STRAPBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 4, hidden_dim)
        )
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.input_projection(x)
        attended, _ = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attended))
        ff_out = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x

class STRAPModel(nn.Module):
    """Modified STRAP model with explicit shape handling."""
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, num_heads=4, 
                 dropout=0.1, forecast_horizon=7):
        super().__init__()
        self.forecast_horizon = forecast_horizon
        
        self.strap_layers = nn.ModuleList([
            STRAPBlock(input_dim if i == 0 else hidden_dim,
                      hidden_dim,
                      num_heads,
                      dropout)
            for i in range(num_layers)
        ])
        
        self.output_layer = nn.Linear(hidden_dim, forecast_horizon)
        
    def forward(self, x):
        # Input shape: (batch_size, lookback, features)
        batch_size = x.shape[0]
        
        # Process through STRAP layers
        for layer in self.strap_layers:
            x = layer(x)
        
        # Global average pooling over temporal dimension
        x = torch.mean(x, dim=1)  # Shape: (batch_size, hidden_dim)
        
        # Project to forecast horizon
        x = self.output_layer(x)  # Shape: (batch_size, forecast_horizon)
        
        return x

def clean_price(price_series):
    """Clean price data without type checking."""
    try:
        if isinstance(price_series.iloc[0], str):
            return pd.to_numeric(price_series.str.replace('$', '').str.replace(',', ''), errors='coerce')
        return pd.to_numeric(price_series, errors='coerce')
    except Exception as e:
        print(f"Error cleaning prices: {e}")
        return pd.to_numeric(price_series, errors='coerce')

def prepare_data(calendar_path, listings_path, n_listings=500):
    """Modified prepare_data function with additional checks."""
    print("\nStarting data preparation...")
    
    try:
        # Load listings data
        print("Loading listings data...")
        listings_df = pd.read_csv(listings_path)
        sampled_listings = listings_df['id'].sample(n=n_listings, random_state=42)
        
        # Load calendar data
        print("Loading calendar data...")
        calendar_df = pd.read_csv(calendar_path)
        calendar_df = calendar_df[calendar_df['listing_id'].isin(sampled_listings)]
        
        print(f"\nInitial data statistics:")
        print(f"Total listings: {len(sampled_listings)}")
        print(f"Total records: {len(calendar_df)}")
        
        # Convert dates
        print("\nProcessing dates...")
        calendar_df['date'] = pd.to_datetime(calendar_df['date'])
        
        # Clean prices
        print("Cleaning price data...")
        if isinstance(calendar_df['price'].iloc[0], str):
            calendar_df['price_numeric'] = pd.to_numeric(
                calendar_df['price'].str.replace('$', '').str.replace(',', ''), 
                errors='coerce'
            )
        else:
            calendar_df['price_numeric'] = calendar_df['price']
        
        # Check for missing values
        missing_prices = calendar_df['price_numeric'].isna().sum()
        print(f"Missing prices: {missing_prices}")
        
        # Remove invalid prices
        calendar_df = calendar_df.dropna(subset=['price_numeric'])
        
        # Sort by listing and date
        print("Sorting data...")
        calendar_df = calendar_df.sort_values(['listing_id', 'date'])
        
        # Create features
        print("\nCreating features...")
        calendar_df['day_of_week'] = calendar_df['date'].dt.dayofweek
        calendar_df['month'] = calendar_df['date'].dt.month
        calendar_df['day_of_year'] = calendar_df['date'].dt.dayofyear
        calendar_df['is_weekend'] = calendar_df['day_of_week'].isin([5, 6]).astype(int)
        calendar_df['week_of_year'] = calendar_df['date'].dt.isocalendar().week
        
        # Merge with listings
        print("Adding listing features...")
        calendar_df = pd.merge(
            calendar_df,
            listings_df[['id', 'latitude', 'longitude']],
            left_on='listing_id',
            right_on='id',
            how='left'
        )
        
        # Define features
        feature_cols = [
            'price_numeric', 
            'day_of_week', 
            'month', 
            'day_of_year',
            'week_of_year',
            'is_weekend',
            'latitude',
            'longitude'
        ]
        
        # Scale features
        print("\nScaling features...")
        feature_scaler = StandardScaler()
        features_scaled = feature_scaler.fit_transform(calendar_df[feature_cols])
        
        # Scale targets
        print("Scaling targets...")
        target_scaler = StandardScaler()
        targets_scaled = target_scaler.fit_transform(calendar_df['price_numeric'].values.reshape(-1, 1))
        
        # Convert to tensors
        print("\nConverting to tensors...")
        features_tensor = torch.FloatTensor(features_scaled)
        targets_tensor = torch.FloatTensor(targets_scaled)
        
        print("\nTensor shapes:")
        print(f"Features: {features_tensor.shape}")
        print(f"Targets: {targets_tensor.shape}")
        
        # Verify data is properly ordered
        print("\nChecking data continuity...")
        date_diffs = calendar_df.groupby('listing_id')['date'].diff().dt.days
        if date_diffs.max() > 1:
            print("Warning: Data has gaps between dates")
        
        return features_tensor, targets_tensor, target_scaler
        
    except Exception as e:
        print(f"\nError in data preparation: {str(e)}")
        raise

def train_model(model, train_loader, val_loader, num_epochs, learning_rate, device):
    """Training function with cleaner output."""
    print("\nStarting model training...")
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    scaler = amp.GradScaler()
    
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'epoch_times': []}
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        model.train()
        train_losses = []
        
        # Update tqdm to refresh less frequently
        train_progress = tqdm(
            train_loader, 
            desc=f'Epoch {epoch+1}/{num_epochs} [Train]',
            ncols=100,  # Fixed width
            mininterval=1.0,  # Update every second
            leave=False  # Don't leave progress bars
        )
        
        for batch_x, batch_y in train_progress:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            with amp.autocast():
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_losses.append(loss.item())
        
        # Validation
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            val_progress = tqdm(
                val_loader, 
                desc=f'Epoch {epoch+1}/{num_epochs} [Val]',
                ncols=100,
                mininterval=1.0,
                leave=False
            )
            
            for batch_x, batch_y in val_progress:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                with amp.autocast():
                    outputs = model(batch_x)
                    val_loss = criterion(outputs, batch_y)
                
                val_losses.append(val_loss.item())
        
        # Calculate metrics
        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        epoch_time = time.time() - epoch_start
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['epoch_times'].append(epoch_time)
        
        # Print epoch summary
        print(f'Epoch [{epoch+1}/{num_epochs}] - {epoch_time:.1f}s')
        print(f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_val_loss,
            }, 'best_model.pth')
            print(f'New best model saved (Val Loss: {best_val_loss:.4f})')
        
        print('-' * 60)  # Add separator between epochs
    
    return history

def main():
    device = select_gpu()
    print(f"Using device: {device}")
    
    try:
        print("\nPreparing data...")
        features, targets, target_scaler = prepare_data(
            r'C:\Users\mvk\Documents\DATA_school\thesis\data_new\paris\paris_merged_calendar.csv',
            r'C:\Users\mvk\Documents\DATA_school\thesis\data_new\paris\2024-06-10\listings.csv'
        )
        
        dataset = TemporalFeatureDataset(
            features, 
            targets,
            lookback=30,
            forecast_horizon=7
        )
        
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        
        train_dataset, val_dataset = torch.utils.data.random_split(
            dataset, 
            [train_size, val_size],
            generator=torch.Generator().manual_seed(42)
        )
        
        print("\nCreating data loaders...")
        train_loader = DataLoader(
            train_dataset, 
            batch_size=32,
            shuffle=True,
            num_workers=0
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=32,
            num_workers=0
        )
        
        print("\nInitializing model...")
        model = STRAPModel(
            input_dim=features.shape[1],
            hidden_dim=128,
            num_layers=2,
            num_heads=4,
            dropout=0.1,
            forecast_horizon=7
        ).to(device)
        
        history = train_model(
            model, 
            train_loader, 
            val_loader, 
            num_epochs=100,
            learning_rate=1e-4,
            device=device
        )
        
        print("\nSaving final model...")
        torch.save({
            'model_state_dict': model.state_dict(),
            'history': history,
            'scaler': target_scaler
        }, 'final_model.pth')
        
        print("Training completed successfully!")
        
    except Exception as e:
        print(f"\nError in execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Found 1 GPUs:
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU

Selecting NVIDIA GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Using device: cuda:0

Preparing data...

Starting data preparation...
Loading listings data...
Loading calendar data...

Initial data statistics:
Total listings: 500
Total records: 339131

Processing dates...
Cleaning price data...
Missing prices: 0
Sorting data...

Creating features...
Adding listing features...

Scaling features...
Scaling targets...

Converting to tensors...

Tensor shapes:
Features: torch.Size([339131, 8])
Targets: torch.Size([339131, 1])

Checking data continuity...

Dataset created with 339095 samples
Lookback: 30, Forecast horizon: 7

Creating data loaders...

Initializing model...

Starting model training...


Epoch [1/100] - 52.6s
Train Loss: 0.1398, Val Loss: 0.1057
New best model saved (Val Loss: 0.1057)
------------------------------------------------------------


Epoch [2/100] - 52.6s
Train Loss: 0.0590, Val Loss: 0.0511
New best model saved (Val Loss: 0.0511)
------------------------------------------------------------


Epoch [3/100] - 52.2s
Train Loss: 0.0494, Val Loss: 0.0501
New best model saved (Val Loss: 0.0501)
------------------------------------------------------------


Epoch [4/100] - 78.0s
Train Loss: 0.0472, Val Loss: 0.0837
------------------------------------------------------------


Epoch [5/100] - 77.5s
Train Loss: 0.0444, Val Loss: 0.0438
New best model saved (Val Loss: 0.0438)
------------------------------------------------------------


Epoch [6/100] - 64.1s
Train Loss: 0.0423, Val Loss: 0.0409
New best model saved (Val Loss: 0.0409)
------------------------------------------------------------


Epoch [7/100] - 68.3s
Train Loss: 0.0392, Val Loss: 0.0343
New best model saved (Val Loss: 0.0343)
------------------------------------------------------------


Epoch [8/100] - 59.8s
Train Loss: 0.0376, Val Loss: 0.0370
------------------------------------------------------------


KeyboardInterrupt: 